In [2]:
from lxml import etree as ET
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib
import multiprocessing
#from multiprocessing import Process
#print("Number of cpu : ", multiprocessing.cpu_count())



In [3]:
%load_ext autoreload
%autoreload 2
config = RClib.loadconfigs('.\config.json')
scandf3 = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
#while 'reconstruct-rccmdpath'.isin(scandf3['processingstate'])] and 'model-rccmdpath'.notin(scandf3['processingstate']) :
    #tobemodeleddf = scandf3['reconstruct-rccmdpath'.isin(scandf3['processingstate']) and 'model-rccmdpath'.notin(scandf3['processingstate']) ]
scandf4 = pd.DataFrame()
for index,scan in scandf3.iterrows():
    scan['rcboxpath'] = Path(scan['RCoutputfolder'] / 'base.rcbox')
    if 'align-rccmdpath' in scan['processingstate'] and scan['rcboxpath'].is_file():
        if not 'commandlist_reconstructsettings' in scan['processingstate']:
            scan['commandlist_reconstructsettings']=Path(config['reconstructsettings'])
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_reconstructsettings')
        if not 'reconstruct-rccmdpath' in scan['processingstate'] :
            scan['commandlist_reconstruct'] = []
            scan['commandlist_reconstruct'].append('-setReconstructionRegion ' + str(scan['rcboxpath']))  
            scan['commandlist_reconstruct'].append('-calculateNormalModel')  
            scan['commandlist_reconstruct'].append('-renameSelectedModel ' + scan['id'] + '_highpoly')    
            scan['commandlist_reconstruct'].append('-save')
            scan['commandlist_reconstruct'].append('-quit')  
            scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_reconstruct', rccmdpathfield='reconstruct-rccmdpath')
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='reconstruct-rccmdpath' )
    scandf4 = scandf4.append(scan)
#pd.write_csv(Path(config['workspace']) / 'rcprocessingdf.csv', sep=';')
scandf4.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
E:\RCprocessing_test\PG_boat91_test\03_RC\commandlist_reconstruct_PG_boat91_test.rccmd


In [ ]:
scandf4 = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
#while 'reconstruct-rccmdpath'.isin(scandf3['processingstate'])] and 'model-rccmdpath'.notin(scandf3['processingstate']) :
    #tobemodeleddf = scandf3['reconstruct-rccmdpath'.isin(scandf3['processingstate']) and 'model-rccmdpath'.notin(scandf3['processingstate']) ]
scandf5 = pd.DataFrame()
for index,scan in scandf4.iterrows():
    scan['rcboxpath'] = Path(scan['RCoutputfolder'] / 'base.rcbox')
    if 'align-rccmdpath' in scan['processingstate'] and scan['rcboxpath'].is_file():
        if not 'commandlist_reconstructsettings' in scan['processingstate']:
            scan['commandlist_reconstructsettings']=Path(config['reconstructsettings'])
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_reconstructsettings')
        if not 'reconstruct-rccmdpath' in scan['processingstate'] :
            scan['commandlist_reconstruct'] = []
            scan['commandlist_reconstruct'].append('-setReconstructionRegion ' + str(scan['rcboxpath']))  
            scan['commandlist_reconstruct'].append('-calculateNormalModel')  
            scan['commandlist_reconstruct'].append('-save')
            scan['commandlist_reconstruct'].append('-quit')  
            scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_reconstruct', rccmdpathfield='reconstruct-rccmdpath')
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='reconstruct-rccmdpath' )
    scandf4 = scandf5.append(scan)
